In [2]:
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< INSTALLATION GUIDE>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 

'''pip install nltk

pip install spacy==2.3.5

pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

pip install pyresparser
pip install -U pyresparser pdfplumber unidecode pandas'''


'pip install nltk\n\npip install spacy==2.3.5\n\npip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz\n\npip install pyresparser\npip install -U pyresparser pdfplumber unidecode pandas'

In [7]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
import base64,random

In [ ]:
import os
import pandas as pd
import base64
from pyresparser import ResumeParser
import pdfplumber
import re
from typing import List, Dict, Optional
import docx2txt  # Thư viện để xử lý file docx và doc
import sys

# ====================== Cấu hình ======================
RESUMES_DIR = 'cv'
OUTPUT_CSV = 'resume_data_update.csv'
UNWANTED_PATTERNS = r'[^\w\s,.!?@#\$%&\*\(\)\-\+=\/\'"–—∶−]'  # Giữ lại các ký tự hợp lệ
SUPPORTED_EXTENSIONS = ['.pdf', '.docx', '.doc']  # Các định dạng file được hỗ trợ
# =======================================================

def preprocess_text(text: str) -> str:
    """
    Chuẩn hóa text từ PDF hoặc DOCX:
    - Xóa các ký tự không mong muốn
    - Chuẩn hóa khoảng trắng
    - Xử lý xuống dòng
    """
    # Thay thế nhiều khoảng trắng và xuống dòng
    text = re.sub(r'\s+', ' ', text)
    # Xóa các ký tự đặc biệt không mong muốn
    text = re.sub(UNWANTED_PATTERNS, '', text)
    # Chuẩn hóa dấu gạch ngang
    text = re.sub(r'[\u2013\u2014\u2212]', '-', text)
    return text.strip()

def extract_name_from_text(text: str) -> Optional[str]:
    """
    Trích xuất tên từ text CV sử dụng regex nâng cao
    """
    # Tìm kiếm mẫu tên ở phần đầu CV
    name_pattern = r'''
        ^                   # Bắt đầu chuỗi
        ([A-ZÀ-Ỹ][a-zà-ỹ]+  # Tên đầu viết hoa
        (?:\s+              # Khoảng trắng
        [A-ZÀ-Ỹ][a-zà-ỹ.]*  # Các từ tiếp theo (cho tên đệm, họ)
        ){1,4})             # Tổng 2-4 từ
        (?=\s*(?:[A-Z]|$))  # Lookahead cho ký tự viết hoa hoặc kết thúc
    '''
    match = re.search(name_pattern, text, re.VERBOSE | re.MULTILINE)
    return match.group(1) if match else None

def extract_text_from_pdf(filepath: str) -> str:
    """
    Trích xuất text từ file PDF
    """
    with pdfplumber.open(filepath) as pdf:
        text = "\n".join([page.extract_text(layout=True) for page in pdf.pages if page.extract_text()])
        return text

def extract_text_from_docx(filepath: str) -> str:
    """
    Trích xuất text từ file DOCX hoặc DOC
    """
    try:
        return docx2txt.process(filepath)
    except Exception as e:
        print(f"Lỗi khi đọc file {filepath}: {str(e)}")
        return ""

def parse_resume(filepath: str) -> Dict:
    """
    Xử lý file PDF hoặc DOCX/DOC và trích xuất thông tin
    """
    try:
        # Kiểm tra loại file dựa trên tên file
        file_lower = filepath.lower()
        
        if file_lower.endswith('.pdf'):
            text = extract_text_from_pdf(filepath)
            file_type = 'pdf'
        elif file_lower.endswith('.docx') or file_lower.endswith('.doc'):
            text = extract_text_from_docx(filepath)
            file_type = 'docx' if file_lower.endswith('.docx') else 'doc'
        else:
            print(f"Không hỗ trợ định dạng file: {filepath}")
            return None
        
        # Làm sạch text
        clean_text = preprocess_text(text)
        
        # Phân tích bằng ResumeParser (cho cả PDF và DOCX)
        parsed_data = ResumeParser(filepath).get_extracted_data()
        
        # Kết hợp dữ liệu
        return {
            'name': extract_name_from_text(clean_text) or parsed_data.get('name'),
            'email': parsed_data.get('email'),
            'mobile_number': parsed_data.get('mobile_number'),
            'skills': parsed_data.get('skills', []),
            'education': parsed_data.get('degree') or parsed_data.get('education', []),
            'experience_years': parsed_data.get('total_experience'),
            'file_path': filepath,
            'raw_text': clean_text
        }
    except Exception as e:
        print(f"Lỗi khi xử lý {filepath}: {str(e)}")
        return None

def check_supported_extension(filename: str) -> bool:
    """
    Kiểm tra xem file có định dạng được hỗ trợ hay không
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in SUPPORTED_EXTENSIONS)

def main():
    # Khởi tạo DataFrame
    resume_data = []
    
    # Kiểm tra thư mục tồn tại
    if not os.path.exists(RESUMES_DIR):
        print(f"Thư mục {RESUMES_DIR} không tồn tại. Vui lòng tạo thư mục này và đặt CV vào.")
        return None
    
    # Duyệt qua các file trong thư mục
    for filename in os.listdir(RESUMES_DIR):
        if check_supported_extension(filename):
            filepath = os.path.join(RESUMES_DIR, filename)
            if data := parse_resume(filepath):
                resume_data.append(data)
                print(f"Đã xử lý: {filename}")
            else:
                print(f"Không thể xử lý: {filename}")
    
    # Tạo DataFrame
    if not resume_data:
        print("Không tìm thấy file CV nào để xử lý.")
        return None
        
    df = pd.DataFrame(resume_data)
    
    # Xử lý cột skills
    df['skills'] = df['skills'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    
    # Sắp xếp cột
    columns_order = [
        'name', 'email', 'mobile_number', 'skills',
        'education', 'experience_years', 'file_path', 'raw_text'
    ]
    df = df[columns_order]
    
    # Lưu kết quả
    df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(f"Đã xử lý xong {len(df)} CV. Kết quả lưu tại: {OUTPUT_CSV}")
    return df

if __name__ == "__main__":
    merged_df = main()
    print(merged_df)

c:\Users\ASUS\.conda\envs\dacn1\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Đã xử lý: -Latest PowerBI Developer_ saiprakash.docx
Đã xử lý: 01-Harisankar Viswanathapillai-PM-SM-2024-20 + Years.docx
Đã xử lý: 1.0.pdf
Đã xử lý: 10+ yrs Prashanth Vedavally.docx
Đã xử lý: 2.0.pdf
Đã xử lý: 2024_Shahid_Ezaz_ Khan_Resume (2).docx
Đã xử lý: 66687eb4f4a2f7421c06ad16 - corptocorp.org.docx
Đã xử lý: A-Manovi-K-Java-full-stack-developer.docx
Đã xử lý: A-Rama-Devi-Full-Stack-Java.docx
Đã xử lý: A-Rohittreddy-Java-Fullstak-dev-1.docx
Đã xử lý: A-Rushi-Reddy-Java.docx
Đã xử lý: A-VINAY-Java-Developer.docx
Đã xử lý: Aalekhya Marella_Full-stack Developer -.docx
Đã xử lý: Aamer Choudhry - Resume.docx
Đã xử lý: Aamir khan Azure Data Engineer (2) (1) (1) (2).docx
Đã xử lý: Aamir-Zeeshan-Resume-1-2.docx
Đã xử lý: Aarif Chittoor.docx
Đã xử lý: Aarsita Resume .docx
Đã xử lý: Aashish Sathaye_.docx
Đã xử lý: AashishK_Sr_Java_Developer-1.docx
Đã xử lý: AashishK_Sr_Java_Developer.docx
Đã xử lý: Aashritha Front end developer(1).docx
Đã xử lý: Aashritha Front end developer.docx
Đã xử lý: 

In [10]:
import pandas as pd
import os
import re

def clean_column_names(df):
    """
    Làm sạch tên cột trong DataFrame:
    - Loại bỏ các cột bắt đầu bằng 'Unnamed'
    - Chuẩn hóa tên cột (lowercase, thay khoảng trắng bằng dấu gạch dưới)
    
    Returns:
        pd.DataFrame: DataFrame đã làm sạch tên cột
    """
    # Loại bỏ cột Unnamed
    unnamed_cols = [col for col in df.columns if isinstance(col, str) and col.startswith('Unnamed')]
    if unnamed_cols:
        df = df.drop(columns=unnamed_cols)
    
    # Chuẩn hóa tên cột
    rename_dict = {}
    for col in df.columns:
        if isinstance(col, str):
            # Thay thế khoảng trắng và ký tự đặc biệt bằng gạch dưới
            new_col = re.sub(r'[^\w\s]', '', col)
            new_col = re.sub(r'\s+', '_', new_col).lower().strip('_')
            rename_dict[col] = new_col
    
    return df.rename(columns=rename_dict)

def merge_cv_datasets(file1_path: str, file2_path: str, output_path: str):
    """
    Hàm merge hai dataset CV và lưu kết quả vào một file CSV mới
    
    Args:
        file1_path (str): Đường dẫn đến file CSV thứ nhất
        file2_path (str): Đường dẫn đến file CSV thứ hai
        output_path (str): Đường dẫn lưu file kết quả
    """
    try:
        # Kiểm tra tồn tại của các file đầu vào
        for file_path in [file1_path, file2_path]:
            if not os.path.exists(file_path):
                print(f"Lỗi: File {file_path} không tồn tại.")
                return None
        
        # Đọc 2 file CSV vào DataFrame và xử lý tên cột
        print(f"Đang đọc file {file1_path}...")
        df1 = pd.read_csv(file1_path)
        original_cols1 = len(df1.columns)
        df1 = clean_column_names(df1)
        print(f"Đã đọc {len(df1)} dòng từ {file1_path}")
        if len(df1.columns) != original_cols1:
            print(f"Đã xử lý tên cột: {original_cols1} -> {len(df1.columns)}")
        
        print(f"Đang đọc file {file2_path}...")
        df2 = pd.read_csv(file2_path)
        original_cols2 = len(df2.columns)
        df2 = clean_column_names(df2)
        print(f"Đã đọc {len(df2)} dòng từ {file2_path}")
        if len(df2.columns) != original_cols2:
            print(f"Đã xử lý tên cột: {original_cols2} -> {len(df2.columns)}")
        
        # Kiểm tra và chuẩn hóa tên cột
        print("Đang kiểm tra cấu trúc dữ liệu...")
        
        # Hiển thị thông tin cột của cả hai DataFrame
        print(f"Các cột trong {file1_path}: {df1.columns.tolist()}")
        print(f"Các cột trong {file2_path}: {df2.columns.tolist()}")
        
        # Ghép 2 DataFrame
        print("Đang ghép dữ liệu...")
        
        # Sử dụng outer join để giữ tất cả các dòng từ cả hai DataFrame
        merged_df = pd.concat([df1, df2], ignore_index=True)
        
        # Loại bỏ duplicates dựa trên email (nếu email tồn tại)
        if 'email' in merged_df.columns:
            # Đếm trước khi loại bỏ duplicates
            before_dedup = len(merged_df)
            
            # Ưu tiên giữ lại bản ghi đầy đủ thông tin hơn (dựa trên số lượng trường không null)
            # Tính số lượng trường không null cho mỗi dòng
            merged_df['non_null_count'] = merged_df.notna().sum(axis=1)
            
            # Sắp xếp theo số trường không null (giảm dần) và loại bỏ duplicates
            merged_df = merged_df.sort_values('non_null_count', ascending=False)
            merged_df = merged_df.drop_duplicates(subset=['email'], keep='first')
            
            # Xóa cột phụ trợ
            merged_df = merged_df.drop(columns=['non_null_count'])
            
            # Thông báo số lượng dòng đã loại bỏ
            after_dedup = len(merged_df)
            print(f"Đã loại bỏ {before_dedup - after_dedup} bản ghi trùng lặp dựa trên email")
            
        # Reset index để đảm bảo index liên tục
        merged_df = merged_df.reset_index(drop=True)
        
        # Làm sạch tên cột của dataset kết quả
        col_count_before = len(merged_df.columns)
        merged_df = clean_column_names(merged_df)
        if col_count_before != len(merged_df.columns):
            print(f"Đã làm sạch tên cột cho dataset kết quả: {col_count_before} -> {len(merged_df.columns)}")
            
        # Lưu kết quả
        print(f"Đang lưu {len(merged_df)} bản ghi vào {output_path}...")
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print(f"Hoàn thành! Dữ liệu đã được lưu vào {output_path}")
        return merged_df
    
    except Exception as e:
        print(f"Đã xảy ra lỗi khi merge datasets: {str(e)}")
        return None

if __name__ == "__main__":
    # Đường dẫn đến các file
    file1_path = "resume_data_update.csv"
    file2_path = "cv.csv"
    output_path = "merged_cv_dataset.csv"
    
    # Thực hiện merge
    merged_df = merge_cv_datasets(file1_path, file2_path, output_path)
    
    # Hiển thị thông tin cơ bản của dataset kết quả
    if merged_df is not None:
        print("\nThông tin dataset kết quả:")
        print(f"Số lượng bản ghi: {len(merged_df)}")
        print(f"Các cột: {merged_df.columns.tolist()}")
        print("\nXem mẫu dữ liệu (5 dòng đầu):")
        print(merged_df.head())

Đang đọc file resume_data_update.csv...
Đã đọc 2757 dòng từ resume_data_update.csv
Đang đọc file cv.csv...
Đã đọc 20 dòng từ cv.csv
Đã xử lý tên cột: 9 -> 8
Đang kiểm tra cấu trúc dữ liệu...
Các cột trong resume_data_update.csv: ['name', 'email', 'mobile_number', 'skills', 'education', 'experience_years', 'file_path', 'raw_text']
Các cột trong cv.csv: ['name', 'email', 'mobile_number', 'skills', 'education', 'experience_years', 'file_path', 'raw_text']
Đang ghép dữ liệu...
Đã loại bỏ 1327 bản ghi trùng lặp dựa trên email
Đang lưu 1450 bản ghi vào merged_cv_dataset.csv...
Hoàn thành! Dữ liệu đã được lưu vào merged_cv_dataset.csv

Thông tin dataset kết quả:
Số lượng bản ghi: 1450
Các cột: ['name', 'email', 'mobile_number', 'skills', 'education', 'experience_years', 'file_path', 'raw_text']

Xem mẫu dữ liệu (5 dòng đầu):
                                        name                     email  \
0                        Durga Sai Prakash M   saiprakashm15@gmail.com   
1  ROOPINI MOHANCHANDE

In [19]:
import pandas as pd
import re
import numpy as np

def is_valid_email(email):
    """
    Kiểm tra xem một email có hợp lệ hay không
    
    Args:
        email: Chuỗi email cần kiểm tra
    
    Returns:
        bool: True nếu email hợp lệ, False nếu không
    """
    # Nếu là NaN hoặc None
    if pd.isna(email):
        return False
    
    # Phải là chuỗi
    if not isinstance(email, str):
        return False
    
    # Loại bỏ email quá dài (có thể là một đoạn văn bản)
    if len(email) > 100:
        return False
    
    # Loại bỏ các chuỗi có quá nhiều ký tự đặc biệt
    special_char_count = len(re.findall(r'[!#$%^&*()+=\[\]{}|\\/:;"<>?]', email))
    if special_char_count > 5:  # Nếu có quá nhiều ký tự đặc biệt
        return False
    
    # Kiểm tra định dạng email cơ bản
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(pattern, email):
        return False
    
    # Kiểm tra xem có phải là đoạn văn bản không (có nhiều từ cách nhau)
    words = email.split()
    if len(words) > 3:  # Nếu có quá nhiều từ
        return False
    
    # Kiểm tra có ít nhất một ký tự @ và một dấu chấm
    if '@' not in email or '.' not in email:
        return False
    
    # Đảm bảo domain có ít nhất 2 ký tự sau dấu chấm cuối cùng
    domain_part = email.split('@')[-1]
    if not domain_part or '.' not in domain_part:
        return False
    
    # Tất cả các điều kiện đã thỏa mãn
    return True

def is_valid_name(name):
    """
    Kiểm tra xem tên có hợp lệ hay không (không chứa ký tự đặc biệt)
    
    Args:
        name: Chuỗi tên cần kiểm tra
    
    Returns:
        bool: True nếu tên hợp lệ, False nếu không
    """
    # Nếu là NaN hoặc None
    if pd.isna(name):
        return True  # Cho phép tên trống
    
    # Phải là chuỗi
    if not isinstance(name, str):
        return False
    
    # Loại bỏ tên quá ngắn
    if len(name.strip()) < 2:
        return False
    
    # Loại bỏ tên quá dài
    if len(name) > 100:
        return False
    
    # Kiểm tra các ký tự đặc biệt không mong muốn
    # Cho phép: chữ cái, số, khoảng trắng, dấu gạch ngang, dấu chấm, dấu phẩy và một số ký tự Unicode phổ biến cho tên tiếng Việt
    pattern = r'[^a-zA-Z0-9\s\-.,àáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳýỷỹỵđÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰỲÝỶỸỴĐ]'
    if re.search(pattern, name):
        return False
    
    # Đảm bảo tên không phải là một chuỗi số hoặc mã
    if re.match(r'^\d+$', name):
        return False
    
    # Đảm bảo tên không chứa các từ khóa không liên quan
    unwanted_keywords = ['undefined', 'null', 'nan', 'none', 'unknown', 'n/a', 'na']
    if name.lower().strip() in unwanted_keywords:
        return False
    
    return True

def is_valid_experience_years(exp):
    """
    Kiểm tra xem số năm kinh nghiệm có hợp lệ hay không (là số)
    
    Args:
        exp: Giá trị năm kinh nghiệm cần kiểm tra
    
    Returns:
        bool: True nếu là số hợp lệ, False nếu không
    """
    # Nếu là None hoặc NaN, cho phép giữ lại
    if pd.isna(exp):
        return True
    
    # Nếu đã là số, kiểm tra phạm vi hợp lý
    if isinstance(exp, (int, float)):
        # Một người khó có thể có hơn 60 năm kinh nghiệm làm việc
        return 0 <= exp <= 60
    
    # Nếu là chuỗi, thử chuyển đổi thành số
    if isinstance(exp, str):
        # Loại bỏ khoảng trắng đầu/cuối
        exp = exp.strip()
        
        # Chuỗi rỗng được coi là NaN
        if not exp:
            return True
        
        # Nếu chuỗi quá dài, có thể là một đoạn text
        if len(exp) > 20:
            return False
        
        # Thử tìm số đầu tiên trong chuỗi
        match = re.search(r'^(\d+(\.\d+)?)', exp)
        if not match:
            return False
        
        # Lấy giá trị số và kiểm tra phạm vi
        try:
            value = float(match.group(1))
            return 0 <= value <= 60
        except ValueError:
            return False
    
    return False

def clean_experience_years(exp):
    """
    Chuẩn hóa trường năm kinh nghiệm thành số
    
    Args:
        exp: Giá trị năm kinh nghiệm cần chuẩn hóa
    
    Returns:
        float: Số năm kinh nghiệm sau khi chuẩn hóa, hoặc NaN nếu không thể chuyển đổi
    """
    if pd.isna(exp):
        return np.nan
    
    if isinstance(exp, (int, float)):
        return float(exp)
    
    if isinstance(exp, str):
        exp = exp.strip()
        if not exp:
            return np.nan
        
        # Lấy số đầu tiên trong chuỗi
        match = re.search(r'^(\d+(\.\d+)?)', exp)
        if match:
            try:
                return float(match.group(1))
            except ValueError:
                return np.nan
    
    return np.nan

def clean_cv_dataset(input_path, output_path):
    """
    Làm sạch dataset CV:
    - Loại bỏ các hàng có email không hợp lệ
    - Loại bỏ các hàng có name chứa ký tự đặc biệt
    - Loại bỏ các hàng có năm kinh nghiệm không phải số
    - Chuẩn hóa trường năm kinh nghiệm
    
    Args:
        input_path (str): Đường dẫn đến file CSV đầu vào
        output_path (str): Đường dẫn lưu file CSV đã làm sạch
    """
    try:
        # Đọc file CSV
        print(f"Đang đọc file {input_path}...")
        df = pd.read_csv(input_path)
        original_rows = len(df)
        print(f"Đã đọc {original_rows} dòng từ {input_path}")
        
        # Tạo bản sao ban đầu để phân tích sự thay đổi
        original_df = df.copy()
        
        # ----- KIỂM TRA VÀ LÀM SẠCH EMAIL -----
        if 'email' in df.columns:
            # Lọc các hàng có email hợp lệ
            valid_email_mask = df['email'].apply(is_valid_email)
            invalid_email_count = (~valid_email_mask).sum()
            
            if invalid_email_count > 0:
                print(f"Phát hiện {invalid_email_count} hàng có email không hợp lệ")
                # Lưu các hàng có email không hợp lệ để kiểm tra
                invalid_emails = df.loc[~valid_email_mask, ['email']].dropna().drop_duplicates()
                print("Các email không hợp lệ (tối đa 10 mẫu):")
                print(invalid_emails.head(10).to_string())
                
                # Lọc ra các hàng có email hợp lệ
                df = df.loc[valid_email_mask]
                print(f"Còn lại {len(df)} hàng sau khi lọc email")
        else:
            print("Cảnh báo: Dataset không có cột 'email'!")
        
        # ----- KIỂM TRA VÀ LÀM SẠCH NAME -----
        if 'name' in df.columns:
            # Lọc các hàng có tên hợp lệ
            valid_name_mask = df['name'].apply(is_valid_name)
            invalid_name_count = (~valid_name_mask).sum()
            
            if invalid_name_count > 0:
                print(f"Phát hiện {invalid_name_count} hàng có tên không hợp lệ")
                # Lưu các hàng có tên không hợp lệ để kiểm tra
                invalid_names = df.loc[~valid_name_mask, ['name']].dropna().drop_duplicates()
                print("Các tên không hợp lệ (tối đa 10 mẫu):")
                print(invalid_names.head(10).to_string())
                
                # Lọc ra các hàng có tên hợp lệ
                df = df.loc[valid_name_mask]
                print(f"Còn lại {len(df)} hàng sau khi lọc tên")
        else:
            print("Cảnh báo: Dataset không có cột 'name'!")
        
        # ----- KIỂM TRA VÀ LÀM SẠCH NĂM KINH NGHIỆM -----
        # Tìm tên cột năm kinh nghiệm
        exp_column = None
        for col_name in ['experience_years', 'exp', 'years_of_experience', 'experience', 'total_experience']:
            if col_name in df.columns:
                exp_column = col_name
                break
        
        if exp_column:
            print(f"Đã tìm thấy cột năm kinh nghiệm: '{exp_column}'")
            
            # Kiểm tra các giá trị không hợp lệ
            valid_exp_mask = df[exp_column].apply(is_valid_experience_years)
            invalid_exp_count = (~valid_exp_mask).sum()
            
            if invalid_exp_count > 0:
                print(f"Phát hiện {invalid_exp_count} hàng có năm kinh nghiệm không hợp lệ")
                # Lưu các giá trị không hợp lệ để kiểm tra
                invalid_exps = df.loc[~valid_exp_mask, [exp_column]].dropna().drop_duplicates()
                print("Các giá trị năm kinh nghiệm không hợp lệ (tối đa 10 mẫu):")
                print(invalid_exps.head(10).to_string())
                
                # Lọc ra các hàng có năm kinh nghiệm hợp lệ
                df = df.loc[valid_exp_mask]
                print(f"Còn lại {len(df)} hàng sau khi lọc năm kinh nghiệm")
            
            # Chuẩn hóa trường năm kinh nghiệm thành số
            df[exp_column] = df[exp_column].apply(clean_experience_years)
        else:
            print("Cảnh báo: Không tìm thấy cột năm kinh nghiệm!")
        
        # ----- PHÂN TÍCH CÁC HÀNG BỊ LOẠI BỎ -----
        if len(original_df) > len(df):
            # Tìm các hàng đã bị loại bỏ
            original_indices = set(original_df.index)
            remaining_indices = set(df.index)
            removed_indices = original_indices - remaining_indices
            
            # Lấy mẫu các hàng bị loại bỏ
            removed_rows = original_df.loc[list(removed_indices)]
            print("\nMẫu các hàng bị loại bỏ (tối đa 5 hàng):")
            print(removed_rows.head().to_string())
        
        # ----- LƯU KẾT QUẢ -----
        # Reset index để đảm bảo index liên tục
        df = df.reset_index(drop=True)
        
        print(f"Đang lưu {len(df)} bản ghi vào {output_path}...")
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        # ----- THỐNG KÊ KẾT QUẢ -----
        removed_rows = original_rows - len(df)
        print(f"\nĐã hoàn thành làm sạch dataset!")
        print(f"Tổng số hàng ban đầu: {original_rows}")
        print(f"Số hàng đã loại bỏ: {removed_rows} ({removed_rows/original_rows*100:.2f}%)")
        print(f"Số hàng còn lại: {len(df)} ({len(df)/original_rows*100:.2f}%)")
        print(f"Dữ liệu đã được lưu vào {output_path}")
        
        return df
    
    except Exception as e:
        print(f"Đã xảy ra lỗi khi làm sạch dataset: {str(e)}")
        # Hiển thị stack trace để debug
        import traceback
        traceback.print_exc()
        return None

if __name__ == "__main__":
    # Đường dẫn đến file CSV
    input_path = "merged_cv_dataset.csv"
    output_path = "cleaned_cv_dataset.csv"
    
    # Thực hiện làm sạch dataset
    cleaned_df = clean_cv_dataset(input_path, output_path)
    
    # Hiển thị thông tin cơ bản của dataset kết quả
    if cleaned_df is not None:
        print("\nThông tin dataset sau khi làm sạch:")
        print(f"Số lượng bản ghi: {len(cleaned_df)}")
        print(f"Các cột: {cleaned_df.columns.tolist()}")
        
        # Hiển thị số lượng giá trị NaN trong mỗi cột
        null_counts = cleaned_df.isnull().sum()
        print("\nSố lượng giá trị NaN trong mỗi cột:")
        print(null_counts)
        
        print("\nXem mẫu dữ liệu (5 dòng đầu):")
        print(cleaned_df.head().to_string())

Đang đọc file merged_cv_dataset.csv...
Đã đọc 1450 dòng từ merged_cv_dataset.csv
Phát hiện 112 hàng có email không hợp lệ
Các email không hợp lệ (tối đa 10 mẫu):
                                                         email
14                                       i.rohitjha@gmail.com,
42                                Email:sai.work1993@gmail.com
46                               sabithaofficial.bk@gmail.com|
67                        E-Mail:Ravikannegundla.dev@gmail.com
77                                   ramyatechwhiz@gmail.com||
87                                          hetal0606.ds@gmail
89                                     chandan.ck02@gmail.com|
140                      manny@ilinksolutions.com/571-348-0804
156                          Shridurga.0193@gmail.com|Memphis,
157  saisri@mastroservices.com|www.linkedin.com/in/shankari-sm
Còn lại 1338 hàng sau khi lọc email
Phát hiện 126 hàng có tên không hợp lệ
Các tên không hợp lệ (tối đa 10 mẫu):
                                

In [ ]:
df

,Unnamed: 0,name,email,mobile_number,skills,education,experience_years,file_path,raw_text
0,0,Le Thanh Long,attentionocr@gmail.com,096-997-3012,"Research, Product owner, Physics, Github, Anal...",[],0.00,cv\1.0.pdf,Le Thanh Long Research and Development Phone09...
1,1,Le Thanh Long,attentionocr@gmail.com,096-997-3012,"Research, Physics, Github, Analyze, Css, Engin...",['MLOPS'],0.00,cv\2.0.pdf,Le Thanh Long Research and Development Birth26...
2,2,NHAN HO,hosybaonhan2004@gmail.com,022-2027,"Research, Os, Documentation, Github, Higher ed...",[],0.00,cv\bao nhan ho sy-cv.pdf,BAO-NHAN HO-SY PYTHON BACK-END DEVELOPER Short...
3,3,thanhduycao |,ctduy20@apcs.fitus.edu.vn,NaN,"Statistics, Training, Apis, Research, Api, Com...",[],1.08,cv\CaoThanhDuy_Resume.pdf,THANH-DUY CAO ctduy20@apcs.fitus.edu.vn thanh...
4,4,NGUYEN THI,chauthi1704@gmail.com,8438728569,"Ui, Access, Apis, Documentation, Css, C++, Use...",[],0.00,cv\ChauThi_resume.pdf,"NGUYEN THI CHAU THI 470, Tran Dai Nghia, Ngu H..."
5,5,Lê Thùy Dương,ltduong2911@gmail.com,0707292696,"Excel, Website, Word, Email, Lan",[],0.00,cv\CV Thuy Duong (Việt)1743071666667.pdf,Lê Thùy Dương Nhân viên văn phòng Ngày sinh 29...
6,6,Dao Khanh Duy,khanhhh.duy@gmail.com,022-2026,"Rest, Javascript, Workflow, Github, Css, User ...",['Bachelor of Science in Computer Science'],1.42,cv\DaoKhanhDuy.pdf,Dao Khanh Duy Sept 2004 0789-402-189 khanhhh...
7,7,Ho Quoc Thien Anh,hoquocthienanh@gmail.com,0905500293,"Plan, Analysis, C++, Communication, Seaborn, D...","[""Bachelor's degree Information Technology""]",2.58,cv\HoQuocThienAnh.pdf,"Ho Quoc Thien Anh Vietnam, Da Nang City hoquo..."
8,8,HUYNH THI,huynhthiphuoc04@gmail.com,022-2026,"Java, Communication, Sql, Docker, Teaching, De...",[],1.25,cv\huynhthiphuoc_resume1.pdf,HUYNH THI PHUOC July 2004 0355-176-782 huynh...
9,9,Nguyen Tien,dungbk.nguyentien@g,0973621049,"Research, Os, C, Jira, Engineering, C++, Agile...",['\uf0b1'],0.00,cv\Nguyen-Tien-Dung.pdf,Nguyen Tien Dung 0973621049 Male Developer ...


In [ ]:
import os
import pandas as pd
from pyresparser import ResumeParser
import pdfplumber
import re

In [ ]:
def pdf_to_base64(file_path):
    #with pdfplumber.open(filepath) as pdf
    with open(file_path, 'rb') as f:
    
        encoded_pdf = base64.b64encode(f.read()).decode('utf-8')

    return encoded_pdf

In [ ]:
encoded_files=[]
for i in df["file_path"].values:
    print()
    encoded_files.append(pdf_to_base64(i))

In [ ]:
df["pdf_to_base64"]=encoded_files

In [ ]:
df.to_csv("updated_cv.csv",index=False)